If Dockerfiles have not been modified, connect to the Jupyter server with:  
- ```http://localhost:8011/tree?token=segment-images-cpu``` to run Torch on CPU  

This notebook describes a pipeline to semantically segment an image with the [OpenMMLab Semantic Segmentation Toolbox and Benchmark](https://github.com/open-mmlab/mmsegmentation).  It takes an input image and saves (1) the inference of the class of each pixel as a ```.json```, (2) a dictionary of classes and their corresponding id as a ```.json```, and (3) a visualisation of the segmented image as a ```.jpg```.

In [5]:
target_dir = "data"
input_dir = "place-pulse-singapore-panos-512-1024"
segmentation_dir = "place-pulse-singapore-segmented-512-1024"
visualisaion_dir = "place-pulse-singapore-segmented-visualised-512-1024"
checkpoint_link = "https://download.openmmlab.com/mmsegmentation/v0.5/fcn/fcn_d6_r101-d16_512x1024_80k_cityscapes/fcn_d6_r101-d16_512x1024_80k_cityscapes_20210308_102747-cb336445.pth"
config_file = "./configs/fcn/fcn-d6_r101-d16_4xb2-80k_cityscapes-512x1024.py"
checkpoint_file = "./fcn_d6_r101-d16_512x1024_80k_cityscapes_20210308_102747-cb336445.pth"

In [6]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
import numpy as np

import json
import os
from pathlib import Path

In [7]:
!wget {checkpoint_link}
model = init_model(config_file, checkpoint_file, device="cpu")

In [8]:
images_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, input_dir)):
    images_path.extend(filenames)
    break
images_id = {}
for image_path in images_path:
    images_id['.'.join(image_path.split('.')[:-1])] = os.path.join(target_dir, input_dir, image_path)

Path(os.path.join(target_dir, segmentation_dir)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(target_dir, visualisaion_dir)).mkdir(parents=True, exist_ok=True)
with open(os.path.join(target_dir, "classes.json"), 'w') as fp:
    json.dump({index: class_name for index, class_name in enumerate(model.dataset_meta["classes"])}, fp)
for id, path in images_id.items():
    if Path(os.path.join(target_dir, segmentation_dir, f"{id}.json")).is_file():
        continue
    image = mmcv.imread(path)
    result = inference_model(model, image)
    output = np.argmax(result.seg_logits.data, axis=0).tolist()
    with open(os.path.join(target_dir, segmentation_dir, f"{id}.json"), 'w') as fp:
        json.dump(output, fp)
    show_result_pyplot(model, image, result, show=True, out_file=os.path.join(target_dir, visualisaion_dir, f"{id}.jpg"), opacity=0.5)  